In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
# Function to format the emails from a directory into a dataframe.
def email2df(dir_path, label):
    email_list = []
    files = os.listdir(dir_path)
    
    for filename in files:
        # Get Text inside file        
        file_path = dir_path + '/' + filename
        if filename and filename != 'cmds':
            f = open(file_path,'r', errors="ignore")
            text_file = f.read()
            f.close()
            
        # Append [Filename (Id), Text, Label] to dataframe
        if filename and text_file:
            email_list.append([filename, text_file, label])
            
    df = pd.DataFrame(email_list, columns=['id', 'text', 'label'])
    return df

In [3]:
# We make a data frame for the ham emails with label 0
df1 = email2df('easy_ham', 0) 
df2 = email2df('easy_ham_2', 0)
df3 = email2df('hard_ham', 0)

# We make a data frame for the spam emails with label 1
df4 = email2df('spam', 1)
df5 = email2df('spam_2', 1)

In [4]:
df = pd.concat([df1, df2, df3, df4, df5],names=['id', 'text', 'label'])

In [5]:
df.shape

(9354, 3)

In [6]:
df.head()

,id,text,label
0,00001.7c53336b37003a9286aba55d2945844c,From exmh-workers-admin@redhat.com Thu Aug 22...,0
1,00002.9c4069e25e1ef370c078db7ee85ff9ac,From Steve_Burt@cursor-system.com Thu Aug 22 ...,0
2,00003.860e3c3cee1b42ead714c5c874fe25f7,From timc@2ubh.com Thu Aug 22 13:52:59 2002\n...,0
3,00004.864220c5b6930b209cc287c361c99af1,From irregulars-admin@tb.tf Thu Aug 22 14:23:...,0
4,00005.bf27cdeaf0b8c4647ecd61b1d09da613,From Stewart.Smith@ee.ed.ac.uk Thu Aug 22 14:...,0


In [7]:
df['label'].value_counts(normalize=True)

0    0.743425
1    0.256575
Name: label, dtype: float64

In [8]:
df['id'].duplicated().sum()

4

In [9]:
df[df['id'].duplicated()]

,id,text,label
1400,cmds,From ilug-admin@linux.ie Wed Dec 4 11:52:36 ...,0
500,cmds,Return-Path: <test-admin@lists.sourceforge.net...,0
1001,cmds,From evtwqmigru@datcon.co.uk Tue Oct 8 11:02...,1
1397,cmds,From ilug-admin@linux.ie Wed Dec 4 11:52:36 ...,1


In [10]:
# Some of the cmds emails got through, so we eliminate them
df.drop_duplicates(subset='id', inplace=True)
print(df['id'].duplicated().sum())

#Check for empty values
print(df.isnull().values.any())

0
False


In [11]:
df.shape

(9350, 3)

In [12]:
from sklearn.utils import shuffle
df = shuffle(df)
df.reset_index(inplace=True, drop=True)

In [13]:
df.head()

,id,text,label
0,01589.be65e620bca0f1734cd13c71aea78d4c,From razor-users-admin@lists.sourceforge.net ...,0
1,0113.ff113297f0ed07536d288c7b2193a8ec,From 2940catv77@alltel.net Tue Aug 27 01:34:0...,1
2,0162.261bb4a36b195f96fcc7da4038b898a3,From Kristian2025161@lancsmail.com Thu Aug 29...,1
3,00260.49cb520f5d726da6f1ec32d0e4d2e38f,From fddgl@data.net.mx Mon Jun 24 17:09:22 20...,1
4,2441.8b6ac53af9cab970afa13ff99ecaf8da,From rssfeeds@jmason.org Mon Oct 7 12:05:32 ...,0


## First test: Without removing email headers

In [75]:
from sklearn.base import BaseEstimator, TransformerMixin
import re

class CleanTextTransformer(BaseEstimator, TransformerMixin):
    """
    A custom Transformer which makes the text in the dataframe lowercase, 
    removes punctuation and numbers.
    """
    def __init__(self, text_split=True):
        self.text_split = text_split
        
    def fit(self, X, y=None):
        return self  # The fit method typically does nothing for transformers
    
    def transform(self, X):
        pattern = r'[0-9\W_]'
        X_transformed = X.copy()
        X_transformed = X_transformed.apply(lambda text: text.lower())
        X_transformed= X_transformed.apply(lambda text: re.sub(pattern, ' ', text))
        if self.text_split:
            print(type(X_transformed))
            print(X_transformed)
            X_transformed = X_transformed.apply(lambda text: text.split())
            print(type(X_transformed))
            print(X_transformed)
        return X_transformed

In [79]:
from sklearn.model_selection import train_test_split

X = df['text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [80]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

# Define the preprocessing for the text column
preprocessor = Pipeline(steps=[
    ('text_cleaning', CleanTextTransformer()),
    ('vectorizer', OneHotEncoder(handle_unknown='ignore'))
])

# Create the full pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

In [81]:
from sklearn.metrics import accuracy_score
# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)
print(X_train.shape, y_train.shape)

# Predict using the pipeline
y_pred = pipeline.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)

<class 'pandas.core.series.Series'>
2031    received  from hawaii ireaye net              ...
993     from fork admin xent com  mon sep             ...
4062    from fork admin xent com  sun sep             ...
8143    from ilug admin linux ie  mon sep             ...
6531    from fork admin xent com  fri jul             ...
                              ...                        
5734    from mail dogma slashnull org  mon jul        ...
5191    from fork admin xent com  thu sep             ...
5390    from rssfeeds jmason org  thu oct             ...
860     from fork admin xent com  fri sep             ...
7270    from webmake talk admin lists sourceforge net ...
Name: text, Length: 6264, dtype: object
<class 'pandas.core.series.Series'>
2031    [received, from, hawaii, ireaye, net, by, linu...
993     [from, fork, admin, xent, com, mon, sep, retur...
4062    [from, fork, admin, xent, com, sun, sep, retur...
8143    [from, ilug, admin, linux, ie, mon, sep, retur...
6531    [from, for

ValueError: Expected 2D array, got 1D array instead:
array=[list(['received', 'from', 'hawaii', 'ireaye', 'net', 'by', 'linux', 'midrange', 'com', 'with', 'esmtp', 'id', 'g', 'l', 'qh', 'for', 'gibbs', 'midrange', 'com', 'sat', 'jul', 'message', 'id', 'g', 'l', 'qh', 'linux', 'midrange', 'com', 'received', 'from', 'alabama', 'by', 'hawaii', 'ireaye', 'net', 'lsmtp', 'for', 'windows', 'nt', 'v', 'b', 'with', 'smtp', 'id', 'a', 'hawaii', 'ireaye', 'net', 'jul', 'date', 'sat', 'jul', 'from', 'callie', 'callie', 'dealsweeps', 'ireaye', 'net', 'subject', 'discover', 'platinum', 'clear', 'card', 'new', 'clear', 'design', 'to', 'gibbs', 'midrange', 'com', 'mime', 'version', 'content', 'type', 'text', 'html', 'x', 'status', 'x', 'keywords', 'doctype', 'html', 'public', 'w', 'c', 'dtd', 'html', 'transitional', 'en', 'meta', 'name', 'ad', 'content', 'ls', 'xi', 'cg', 'html', 'head', 'title', 'discover', 'reg', 'platinum', 'clear', 'card', 'title', 'head', 'map', 'name', 'apply', 'area', 'shape',

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('clean_text', CleanTextTransformer(text_split=False)),
    ('vectorizer', TfidfVectorizer()),
    ('classifier', LogisticRegression())
])

# Train the pipeline
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

<class 'pandas.core.series.Series'>
False
<class 'pandas.core.series.Series'>
False


In [53]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred)
print(acc)

0.9834224598930481


In [ ]:
# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)
print(X_train.shape, y_train.shape)

# Predict using the pipeline
y_pred = pipeline.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

# Splitting the data
X = df['text']
y = df['label']

# Instantiate the custom transformer and vectorizer
cleaner = CleanTextTransformer(text_split=True)
countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english')
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')

# Apply the custom transformer
X_cleaned = cleaner.fit_transform(X)

<class 'pandas.core.series.Series'>


In [39]:
# Ensure the input to TfidfVectorizer is a 1D array of strings
X_vectorized = countvectorizer.fit_transform(X_cleaned)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.33, random_state=42)


In [40]:
# Fit a KNeighborsClassifier as an example
model = KNeighborsClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

NameError: name 'KNeighborsClassifier' is not defined

In [ ]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred)
print(acc)